In [55]:
import pandas as pd
import numpy as np
import pathlib
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [12]:
results = pathlib.Path('/home/jakub/Studia/inż/Semestr7/thesis/kod/OneDrive_NEW_GCONET/result_all_7_methods.csv')

In [73]:
df = pd.read_csv(results)
print(f"Wielkość dataframe: {len(df)}")
print(f"Nagłówki danych: {[row for row in df.columns]}")

Wielkość dataframe: 560
Nagłówki danych: ['data_class', 'method', 'mae', 'max-fm', 'mean-fm', 'max-Emeasure', 'mean-Emeasure', 'S-measure', 'AP', 'AUC']


In [74]:
df = df[df.method != 'CoU2Net']
df = df[df.method != 'U2Net']
df = df[df.method != 'LDF']

In [75]:
df.head()

,data_class,method,mae,max-fm,mean-fm,max-Emeasure,mean-Emeasure,S-measure,AP,AUC
1,CoCA_beer_bottle,CoLDF,0.0561,0.6499,0.6422,0.8297,0.8168,0.7820,0.5994,0.8856
2,CoCA_beer_bottle,COEGNET,0.1174,0.5072,0.4822,0.7089,0.6964,0.6305,0.4033,0.7684
3,CoCA_beer_bottle,GICD,0.0894,0.5814,0.5759,0.7996,0.7874,0.7006,0.4852,0.8361
4,CoCA_beer_bottle,GCoNet,0.0946,0.5122,0.5017,0.8219,0.7909,0.6725,0.3970,0.8001
8,CoCA_high_heels,CoLDF,0.1314,0.5465,0.5402,0.7856,0.7731,0.6793,0.4670,0.8513


In [76]:
print("Inicjalizacja słownika wynikowego")

best_dict = {
    method: list() for method in df.method.unique()
}
# print(best_dict)

print("Wypełnianie słownika wynikowego")

for dc in df.data_class.unique():
    temp_df = df.loc[df.data_class == dc]
    temp_best_dict = {
        method: 0 for method in df.method.unique()
    }
    min_keys = ['mae']
#     min_keys = [' max-fm', ' mean-fm', ' max-Emeasure', ' mean-Emeasure', ' S-measure', ' AP', ' AUC']
    for key in min_keys:
        best = temp_df[temp_df[key] == temp_df.loc[:, key].min()]['method'].values[0]
        temp_best_dict[best] = temp_best_dict[best] + 1
#     max_keys = [' mae']
    max_keys = ['max-fm', 'mean-fm', 'max-Emeasure', 'mean-Emeasure', 'S-measure', 'AP', 'AUC']
#     max_keys = [' max-fm', ' max-Emeasure', ' S-measure']
    for key in max_keys:
        best = temp_df[temp_df[key] == temp_df.loc[:, key].max()]['method'].values[0]
        temp_best_dict[best] = temp_best_dict[best] + 1
    my_max = max(temp_best_dict, key=temp_best_dict.get)
#     for item in temp_best_dict.items():
#         if my_max == item[1]:
#             best_key = item[0]
    name = dc.replace('CoCA_', '')
#     print(best_dict[best_key].append(name))
    best_dict[my_max].append(name)

        
pp.pprint(best_dict)

Inicjalizacja słownika wynikowego
Wypełnianie słownika wynikowego
{   'COEGNET': [   'chopsticks',
                   'sticky_note',
                   'tablet',
                   'frisbee',
                   'watermelon',
                   'handbag',
                   'gift_box',
                   'teddy_bear',
                   'dice',
                   'doughnut',
                   'Yellow_duck',
                   'rolling_pin',
                   'clover',
                   'glove',
                   'pillow'],
    'CoLDF': [   'beer_bottle',
                 'sunflower',
                 'glasses',
                 'pigeon',
                 'hammock',
                 'UAV',
                 'Accordion',
                 'faucet',
                 'hourglass',
                 'stool',
                 'macaroon',
                 'wheelchair',
                 'stethoscope',
                 'pineapple',
                 'headphone',
                 'fishing_rod',
  

In [49]:
# print(best_dict)

{'CoU2Net': [], 'CoLDF': ['beer_bottle', 'high_heels', 'chopsticks', 'squirrel', 'poker', 'rabbit', 'sunflower', 'sticky_note', 'glasses', 'pigeon', 'violin', 'pine_cone', 'helicopter', 'hammock', 'camera', 'tablet', 'backpack', 'hammer', 'potato', 'UAV', 'Accordion', 'globe', 'faucet', 'pocket_watch', 'microphone', 'lollipop', 'towel', 'eggplant', 'hourglass', 'frisbee', 'rocking_horse', 'strawberry', 'watermelon', 'dumbbell', 'handbag', 'stool', 'thermometer', 'dolphin', 'macaroon', 'binoculars', 'gift_box', 'wheelchair', 'teddy_bear', 'toy_car', 'persimmon', 'stethoscope', 'guitar', 'dice', 'roller-skating', 'doughnut', 'pineapple', 'headphone', 'baseball', 'hat', 'soap_bubble', 'pumpkin', 'fishing_rod', 'belt', 'moon', 'harp', 'Yellow_duck', 'alarm_clock', 'watering_can', 'tomato', 'minions', 'calculator', 'ice_cream', 'rolling_pin', 'avocado', 'clover', 'candle', 'typewriter', 'glove', 'pillow', 'whisk', 'camel', 'waffles', 'boots', 'butterfly', 'key', 'beer_bottle', 'high_heels',

In [99]:
# # NOT WORKING
# for i in range (0, len(df) - 3, 4):
#     temp_df = df.loc[i:i + 3, :]
#     temp_best_dict = {
#         'GICD': 0,
#         'COEGNET': 0,
#         'CoU2Net': 0,
#         'GCoNet': 0
#     }
# #     min_keys = [' max-fm', ' mean-fm', ' max-Emeasure', ' mean-Emeasure', ' S-measure', ' AP', ' AUC']
#     min_keys = [' max-fm', ' max-Emeasure'' S-measure']
#     for key in min_keys:
#         best = temp_df[temp_df[key] == temp_df.loc[:, key].min()]['data (method)'].tolist()[0].split(' ')[-1].replace('(', '').replace(')', '')
#         temp_best_dict[best] = temp_best_dict[best] + 1
#     max_keys = [' mae']
#     for key in max_keys:
#         best = temp_df[temp_df[key] == temp_df.loc[:, key].max()]['data (method)'].tolist()[0].split(' ')[-1].replace('(', '').replace(')', '')
#         temp_best_dict[best] = temp_best_dict[best] + 1
#     my_max = max(temp_best_dict.values())
#     for item in temp_best_dict.items():
#         if my_max == item[1]:
#             best_key = item[0]
#     name = temp_df['data (method)'].tolist()[0].split(' ')[0].replace('CoCA_', '')
# #     print(best_dict[best_key].append(name))
#     best_dict[best_key].append(name)


In [77]:
best_num_dict = {
    method: len(best_dict[method]) for method in best_dict.keys()
}

pp.pprint(best_num_dict)

{'COEGNET': 15, 'CoLDF': 23, 'GCoNet': 28, 'GICD': 14}


In [62]:
my_max = max(temp_best_dict.values())

In [63]:
for item in temp_best_dict.items():
    if my_max == item[1]:
        best_key = item[0]

In [64]:
best_key

'COEGNET'

In [68]:
temp_best_dict

{'GICD': 0, 'COEGNET': 2, 'CoU2Net': 6, 'GCoNet': 0}

In [69]:
temp_df

,data (method),mae,max-fm,mean-fm,max-Emeasure,mean-Emeasure,S-measure,AP,AUC
316,CoCA_key (CoU2Net),0.1195 mae,0.3674 max-fm,0.3470 mean-fm,0.6531 max-Emeasure,0.6263 mean-Emeasure,0.6214 S-measure,0.3052 AP,0.8062 AUC
317,CoCA_key (COEGNET),0.0890 mae,0.4688 max-fm,0.4513 mean-fm,0.6986 max-Emeasure,0.6686 mean-Emeasure,0.6191 S-measure,0.3259 AP,0.6994 AUC
318,CoCA_key (GICD),0.0898 mae,0.5274 max-fm,0.5117 mean-fm,0.7681 max-Emeasure,0.7290 mean-Emeasure,0.7154 S-measure,0.4648 AP,0.8813 AUC
319,CoCA_key (GCoNet),0.0593 mae,0.6236 max-fm,0.5954 mean-fm,0.8354 max-Emeasure,0.8036 mean-Emeasure,0.7258 S-measure,0.5557 AP,0.8941 AUC
